# Lesson 2: A real-world prompt

前のレッスンでは、重要なプロンプト作成のヒントをいくつか学び、それぞれを単体で使う例を見ました。ここでは、それらのテクニックを複数組み合わせた、より大きなプロンプトを書いてみましょう。

---


## Our prompting goal

このレッスンでは、長い診療記録（medical record）を入力として受け取り、医師が次回の診察に備えるために役立つ重要情報を含む要約を生成する「診療記録サマライザ」プロンプトを書きます。


各患者の診療記録は次のような形式です：

```
患者名: イブリン・トンプソン
年齢: 78歳
診療記録:

1985年: 2型糖尿病と診断、メトホルミン開始
1992年: 高血圧を発症、リシノプリル処方
1998年: 変形性関節症により右股関節全置換術
2000年: 甲状腺機能低下症と診断、レボチロキシン開始
2003年: 白内障手術（両眼）
2005年: 心房細動で入院、ワルファリン開始
2008年: ビタミンB12欠乏症と診断、月1回の注射開始
2010年: A1C値上昇によりメトホルミン増量
2011年: 一過性脳虚血発作（TIA）で入院、アスピリンを追加
2013年: ステージ2の乳がんと診断、乳房温存術と放射線治療
2014年: 乳がん再発予防のためアナストロゾール開始
2015年: 慢性腎臓病（CKD）ステージ3を発症、メトホルミン調整
2017年: 変形性関節症により左膝関節全置換術
2018年: 肺炎で入院、静脈内抗生物質で治療
2019年: 軽度認知障害が認められ、ドネペジル開始
2020年: 難治性高血圧によりリシノプリル増量
2021年: 再発性尿路感染症、低用量予防的抗生物質処方
2022年: 年次マンモグラフィーは異常なし、eGFRで腎機能悪化を示す
2023年: 移動能力の低下、理学療法と訪問介護開始
```

最終目標は、医療者が診察前に素早く把握できるよう、**一貫したフォーマット** のサマリーを生成することです。各サマリーには、少なくとも次の要素を含めたいです：

* 患者名
* 年齢
* 主要な診断（diagnoses）
* 処方薬（medications）の一覧
* 最近の懸念点（recent concerns）
* 医師向けのアクション項目（action items）

例えば、上の診療記録に対して期待する出力は次のような形です（例）：

```
患者名: イブリン・トンプソン
年齢: 78歳

主要な診断:
- 2型糖尿病 (1985年)
- 高血圧 (1992年)
- 変形性関節症 (1998年と2017年に股関節・膝関節置換術)
- 甲状腺機能低下症 (2000年)
- 心房細動 (2005年)
- ビタミンB12欠乏症 (2008年)
- 一過性脳虚血発作（TIA） (2011年)
- 乳がん (2013年)
- 慢性腎臓病（CKD）ステージ3 (2015年)
- 肺炎 (2018年)
- 軽度認知障害 (2019年)
- 再発性尿路感染症（UTI） (2021年)

処方薬:
- メトホルミン (糖尿病)
- リシノプリル (高血圧)
- レボチロキシン (甲状腺機能低下症)
- ワルファリン (心房細動)
- アスピリン (抗血小板)
- アナストロゾール (乳がん再発予防)
- ドネペジル (認知障害)
- 低用量予防的抗生物質 (再発性UTI)

その他の治療:
- 股関節全置換術 (1998年)
- 白内障手術 (2003年)
- ビタミンB12注射 (2008年)
- 乳房温存術と放射線治療 (乳がん、2013年)
- 膝関節全置換術 (2017年)
- 理学療法と訪問介護 (2023年)

最近の懸念点:
- 腎機能の悪化 (2022年のeGFR低下)
- 移動能力の低下 (2023年)

アクション項目:
- 腎機能をモニターし、必要に応じて薬剤を調整
- 理学療法と訪問介護の継続
- 認知機能低下を評価し、治療計画を調整
- 移動の問題と転倒リスクに対処
- 推奨されるがん検診の遵守を確保
```


ここでは、5件の診療記録を含む Python リストを用意し、このプロンプトで試します：


In [ ]:
patient_records = [
    """
患者名: イブリン・トンプソン
年齢: 78歳
診療記録:

1985年: 2型糖尿病と診断、メトホルミン開始
1992年: 高血圧を発症、リシノプリル処方
1998年: 変形性関節症により右股関節全置換術
2000年: 甲状腺機能低下症と診断、レボチロキシン開始
2003年: 白内障手術（両眼）
2005年: 心房細動で入院、ワルファリン開始
2008年: ビタミンB12欠乏症と診断、月1回の注射開始
2010年: A1C値上昇によりメトホルミン増量
2011年: 一過性脳虚血発作（TIA）で入院、アスピリンを追加
2013年: ステージ2の乳がんと診断、乳房温存術と放射線治療
2014年: 乳がん再発予防のためアナストロゾール開始
2015年: 慢性腎臓病（CKD）ステージ3を発症、メトホルミン調整
2017年: 変形性関節症により左膝関節全置換術
2018年: 肺炎で入院、静脈内抗生物質で治療
2019年: 軽度認知障害が認められ、ドネペジル開始
2020年: 難治性高血圧によりリシノプリル増量
2021年: 再発性尿路感染症、低用量予防的抗生物質処方
2022年: 年次マンモグラフィーは異常なし、eGFRで腎機能悪化を示す
2023年: 移動能力の低下、理学療法と訪問介護開始
    """,
    """
患者名: マーカス・レイエス
年齢: 42歳
診療記録:

2001年: 全般性不安障害（GAD）と診断、パロキセチン開始
2003年: 大うつ病性障害（MDD）と診断、ブプロピオン追加
2005年: 自殺念慮で入院、認知行動療法（CBT）を追加
2007年: 注意欠如・多動性障害（ADHD）と診断、メチルフェニデート開始
2009年: パロキセチンの副作用を報告、エスシタロプラムに変更
2012年: 閉塞性睡眠時無呼吸症候群（OSA）と診断、CPAP療法開始
2014年: 高血圧と診断、ロサルタン開始
2015年: 体重増加が認められ、栄養士に紹介
2016年: 2型糖尿病と診断、メトホルミン開始
2017年: 糖尿病性ケトアシドーシス（DKA）で入院、インスリン療法開始
2018年: 悪夢を報告、ブプロピオンからベンラファキシンに変更
2019年: 胃食道逆流症（GERD）と診断、オメプラゾール開始
2020年: 離婚、セラピーセッションを増やし、弁証法的行動療法（DBT）を追加
2021年: 足底筋膜炎を発症、装具と理学療法を処方
2022年: パニック発作で入院、心筋梗塞と誤認されたが心臓検査は陰性
2023年: 自殺未遂、30日間の入院精神科治療
2023年: 退院後、新しい抗精神病薬（クエチアピン）と気分安定薬（ラモトリギン）を開始
2024年: 気分と睡眠の改善を報告、体重減少が認められる
2024年: A1C値が改善、インスリン用量を減量
    """,
    """
患者名: リリー・チェン
年齢: 8歳
診療記録:

2016年（出生時）: 34週で出生、ファロー四徴症（TOF）と診断
  - 肺血流増加のためシャントを即座に設置する手術
2016年（3ヶ月）: 心エコーで右心室肥大の悪化を示す
2017年（8ヶ月）: TOFの完全修復（VSD閉鎖、肺動脈弁置換、右室流出路修復）
2017年（10ヶ月）: 術後不整脈を発症、アミオダロン開始
2018年（14ヶ月）: 発達遅延が認められ、早期介入サービスに紹介
2018年（18ヶ月）: 言語発達遅延のため言語療法を開始
2019年（2歳）: 成長障害と診断、高カロリー食を開始
2019年（2.5歳）: 微細運動技能の遅延のため作業療法を開始
2020年（3歳）: 心臓カテーテル検査で軽度の肺動脈狭窄を示す
2020年（3.5歳）: 感覚処理障害（SPD）と診断
2021年（4歳）: IEP（個別教育計画）付き統合保育プログラムを開始
2021年（4.5歳）: RSV細気管支炎で入院、短期間の酸素補助が必要
2022年（5歳）: 心エコーで肺動脈狭窄の進行を示し、バルーン弁形成術を実施
2022年（5.5歳）: 注意欠如・多動性障害（ADHD）と診断、行動療法を開始
2023年（6歳）: 感音難聴のため人工内耳手術
2023年（7歳）: IEPサポートを継続しながら通常学校を開始
2024年（7.5歳）: 作業療法を終了、微細運動技能の目標を達成
2024年（8歳）: 定期的な心臓フォローアップで肺動脈弁機能は安定
2024年（8歳）: 言語発達は順調に進行、継続的な療法
    """,
    """
患者名: ジェイソン・トラン
年齢: 25歳
診療記録:

2010年（11歳）: 喘息と診断、アルブテロール吸入器開始
2012年（13歳）: 初めてのフットボールによる脳震盪、一時的な意識喪失
2013年（14歳）: サッカー中に右脛骨骨折、外科的固定
2014年（15歳）: 2回目の脳震盪、脳震盪後症候群を発症
  - 症状: 頭痛、めまい、記憶障害
  - 小児神経科医に紹介、認知リハビリテーション療法
2015年（16歳）: 不安と抑うつを発症、フルオキセチン開始
2016年（17歳）: バスケットボール中にACL断裂（左膝）、再建手術
  - 6ヶ月のリハビリ、非接触スポーツに変更
2017年（18歳）: 高校卒業、学術奨学金で大学入学
2018年（19歳）: スポーツ外傷に関連するPTSDと診断
  - 認知行動療法（CBT）を開始
2019年（20歳）: バスケットボールに復帰を試みたが、パニック発作を経験
  - セラピーセッションを増やし、暴露療法を追加
2020年（21歳）: COVID-19パンデミック、リモート学習、不安の増加を報告
  - マインドフルネス瞑想とヨガを開始
2021年（22歳）: 睡眠時無呼吸と診断、CPAP療法開始
  - 睡眠検査で脳震盪と睡眠障害の関連が示唆される
2022年（23歳）: 低負荷スポーツ（水泳、サイクリング）に徐々に復帰
  - 気分と睡眠の質の改善を報告
2023年（24歳）: 大学卒業、スポーツ分析の仕事を開始
  - セラピーを継続、現在は隔週
  - 若年層の脳震盪啓発プログラムにボランティア参加
2024年（25歳）: 年次健康診断 - 喘息は良好にコントロール、精神健康は安定
  - 過去2年間でスポーツ関連の外傷なし
  - 初めてのハーフマラソンに向けてトレーニング中
    """,
    """
患者名: アミラ・カーン
年齢: 36歳
診療記録:

2011年: 再発性の関節痛と疲労、初期診断は線維筋痛症
  - プレガバリンと理学療法を開始
2012年: 持続的な発疹と光線過敏症を発症
  - 皮膚科医が皮膚エリテマトーデス（CLE）と診断
  - 局所コルチコステロイドと日焼け止めを開始
2013年: 重度の疲労、脱毛、認知障害（「ブレインフォグ」）を訴える
  - 血液検査でANA陽性と抗dsDNA抗体陽性を示す
  - 全身性エリテマトーデス（SLE）と診断
  - ヒドロキシクロロキンと低用量プレドニゾンを開始
2014年: ループス腎炎（クラスIII）で入院
  - 腎生検で診断を確認
  - ミコフェノール酸モフェチルとプレドニゾン増量を開始
2015年: SLEに続発する間質性肺疾患（ILD）を発症
  - シクロホスファミドパルス療法を開始
  - 在宅酸素療法が必要
2016年: 続発性シェーグレン症候群と診断
  - 症状: ドライアイ、口渇
  - ピロカルピンと人工涙液を開始
2017年: ループス脳炎で入院
  - 症状: けいれん、混乱、記憶喪失
  - MRIで脳の炎症を示す
  - 高用量ステロイドとリツキシマブで治療
2018年: ステロイド誘発性の股関節無菌性骨壊死（AVN）を発症
  - 股関節全置換術を受ける
2019年: フレア頻度を減らすためベリムマブを開始
  - プレドニゾン用量を徐々に減量
2020年: COVID-19パンデミック、免疫抑制のため自己隔離
  - テレメディシンフォローアップ、在宅点滴
2021年: ステロイド誘発性糖尿病と診断
  - メトホルミンとインスリンを開始
  - 内分泌科医と栄養士に紹介
2022年: ループスのフレア、関節痛と疲労の増加
  - 薬剤を調整: ミコフェノール酸を増量、アバタセプトを追加
2023年: 長期ステロイド使用による心血管スクリーニング
  - 心エコーで拡張機能不全の初期徴候を示す
  - ACE阻害薬を開始、心臓リハビリに紹介
2024年: すべての臓器系で改善が認められる
  - 免疫抑制薬を減量中、慎重にモニタリング
  - 理学療法と関節の健康のためのヨガを継続
  - 妊娠を計画、ハイリスク産科に相談
    """
]

まずは1件だけ表示して、リストが期待どおりに扱えていることを確認します：


In [8]:
patient_records[2]

'\nPatient Name: Lily Chen\nAge: 8\nMedical Record:\n\n2016 (Birth): Born at 34 weeks, diagnosed with Tetralogy of Fallot (TOF)\n  - Immediate surgery to place a shunt for increased pulmonary blood flow\n2016 (3 months): Echocardiogram showed worsening right ventricular hypertrophy\n2017 (8 months): Complete repair of TOF (VSD closure, pulmonary valve replacement, RV outflow tract repair)\n2017 (10 months): Developed post-operative arrhythmias, started on amiodarone\n2018 (14 months): Developmental delay noted, referred to early intervention services\n2018 (18 months): Speech therapy initiated for delayed language development\n2019 (2 years): Diagnosed with failure to thrive, started on high-calorie diet\n2019 (2.5 years): Occupational therapy started for fine motor skill delays\n2020 (3 years): Cardiac catheterization showed mild pulmonary stenosis\n2020 (3.5 years): Diagnosed with sensory processing disorder (SPD)\n2021 (4 years): Started integrated preschool program with IEP (Indivi

---

## Our initial "bad" prompt

まずは、Claude に要約してもらうだけの、非常に単純な（＝良くない）プロンプトから始めます。最初の試みとしては例えば次のようなものです：

> I have this patient medical record. Can you summarize it for me?
> 
> {medical record goes here}
> 
> I need this for a quick review before the patient's appointment tomorrow.

これを「診療記録を動的に差し込める」形で、具体的なプロンプトとして定式化してみましょう。


In [ ]:
initial_prompt = """
この診療記録があります。要約していただけますか？

{record}

明日の患者の診察前に素早く確認するために必要です。
"""

次に、1件の診療記録を受け取り、プロンプトに差し込み、Claude に要約させる簡単な関数を書きます：


In [98]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()
client = Anthropic()

def generate_summary_with_bad_prompt(patient_record):
    prompt_with_record = initial_prompt.format(record=patient_record)
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        messages=[{"role": "user", "content": prompt_with_record}]
    )
    print("===============================")
    print(response.content[0].text)

`patient_records` にある5件すべての診療記録で試してみましょう：


In [16]:
generate_summary_with_bad_prompt(patient_records[0])

Here is a summary of Evelyn Thompson's 78-year-old medical record:

Chronic Conditions:
- Type 2 diabetes (since 1985) - on metformin, dose increased in 2010
- Hypertension (since 1992) - on lisinopril, dose increased in 2020
- Hypothyroidism (since 2000) - on levothyroxine  
- Atrial fibrillation (since 2005) - on warfarin
- Vitamin B12 deficiency (since 2008) - receiving monthly injections
- Chronic kidney disease stage 3 (since 2015) - metformin adjusted
- Mild cognitive impairment (since 2019) - on donepezil

Surgical History:
- Total hip replacement (1998) - right side, due to osteoarthritis
- Cataract surgery (2003) - both eyes
- Lumpectomy and radiation (2013) - for stage 2 breast cancer  
- Total knee replacement (2017) - left side, due to osteoarthritis

Cancer History: 
- Breast cancer (2013) - currently on anastrozole for recurrence prevention

Recent Issues:
- Recurrent UTIs (2021) - on prophylactic antibiotics
- Worsening kidney function per eGFR (2022)
- Declining mobilit

In [17]:
generate_summary_with_bad_prompt(patient_records[1])

Here's a summary of Marcus Reyes' (age 42) medical record:

Mental Health:
- Diagnosed with generalized anxiety disorder in 2001, treated with paroxetine
- Major depressive disorder diagnosed in 2003, bupropion added
- Hospitalized for suicidal ideation in 2005, started cognitive behavioral therapy (CBT)  
- ADHD diagnosis in 2007, started methylphenidate
- Medications changed over time due to side effects/inefficacy (escitalopram, venlafaxine, quetiapine, lamotrigine)
- Attempted suicide in 2023, required 30-day inpatient psychiatric treatment
- Divorce in 2020 led to increased therapy (added dialectical behavior therapy)

Other Conditions:
- Obstructive sleep apnea in 2012, started CPAP therapy
- Hypertension in 2014, started losartan  
- Weight gain, referred to nutritionist in 2015
- Type 2 diabetes diagnosed in 2016, metformin started
- Hospitalized for diabetic ketoacidosis in 2017, insulin initiated
- GERD diagnosed in 2019, started omeprazole
- Plantar fasciitis in 2021, prescr

In [18]:
generate_summary_with_bad_prompt(patient_records[2])

Here is a summary of Lily Chen's 8-year medical record:

Lily was born prematurely at 34 weeks in 2016 with Tetralogy of Fallot (TOF), a congenital heart defect. She underwent immediate surgery for a shunt and then complete TOF repair at 8 months old in 2017. Post-operatively, she developed arrhythmias requiring medication.

Starting around 1 year old, Lily experienced developmental delays - failure to thrive, speech/language delays requiring therapy, fine motor skill delays requiring occupational therapy, and a sensory processing disorder diagnosis. An IEP was started in preschool for extra support.

Cardiac issues continued with mild pulmonary stenosis at 3 years old, requiring a balloon valvuloplasty at 5 years. She was also diagnosed with ADHD at 5.5 years and started behavioral therapy.

Other medical events included an RSV bronchiolitis hospitalization at 4.5 years requiring oxygen, and cochlear implant surgery at 6 years for sensorineural hearing loss.

Now at 8 years old, Lily 

In [19]:
generate_summary_with_bad_prompt(patient_records[3])

Here is a summary of Jason Tran's 25-year-old medical record:

Asthma & Sports Injuries
- Diagnosed with asthma at age 11, prescribed albuterol inhaler
- Multiple sports concussions starting at age 13, with post-concussion syndrome 
- Fracture and knee injuries requiring surgery in teens

Mental Health
- Developed anxiety and depression at 16, started fluoxetine
- Diagnosed with PTSD at 19 related to sports injuries, started CBT and exposure therapy
- Experienced panic attack at 20 trying to return to basketball
- Increased anxiety during COVID-19 pandemic at 21, started mindfulness practices

Sleep Issues
- Diagnosed with sleep apnea at 22 potentially linked to concussions, started CPAP

Recent Updates
- Gradual return to low-impact sports at 23 with improved mood/sleep
- Graduated college at 24, working in sports analytics 
- Continuing biweekly therapy and youth concussion awareness program
- Training for first half-marathon at 25, asthma and mental health stable

Key Issues
- Histo

In [20]:
generate_summary_with_bad_prompt(patient_records[4])

Here is a summary of Amira Khan's lengthy and complex medical history:

Amira Khan is a 36-year-old woman with a long-standing diagnosis of Systemic Lupus Erythematosus (SLE). Her initial symptoms in 2011 were joint pain and fatigue. After developing rashes and photosensitivity in 2012, she was diagnosed with cutaneous lupus and started on topical steroids. In 2013, positive autoantibodies and multi-system involvement led to the SLE diagnosis.

Over the years, Amira has experienced numerous lupus complications affecting multiple organs:
- Lupus nephritis requiring immunosuppressants (2014)
- Interstitial lung disease requiring cyclophosphamide and oxygen (2015)  
- Secondary Sjogren's syndrome (2016)
- Lupus cerebritis with brain inflammation (2017)
- Avascular necrosis of hip requiring joint replacement (2018)

She has required intensive treatment with hydroxychloroquine, mycophenolate, cyclophosphamide, rituximab, belimumab, and long-term corticosteroids leading to side effects like 

Claude が生成した要約をよく見ると、出力がバラバラです。長い段落だけのものもあれば、箇条書きのものもあります。内容も記録ごとに大きく揺れています。内容が致命的に悪いわけではありませんが、フォーマット・長さ・含まれる項目が一貫しておらず、プログラムで扱うのが難しいです。改善しましょう！

---


## Improving the prompt

前のレッスンで学んだように、より良く一貫した結果を得るために使えるテクニックがいくつかあります。例えば：

* **明確な system prompt** を追加し、医療従事者や医療データ分析者など、アシスタントの文脈と役割を定義する
* 入力データを XML タグ（例：`<patient_record>`）で囲み、どこが入力データなのかを明確にする
* 要約で何に注目すべきか（診断、薬、最近の懸念など）を具体的に指示する
* 出力の望ましい形式（箇条書きや固定セクション構造など）を指定する
* 良い例（well-formatted summary）を提示して出力をガイドする
* 出力も XML タグで構造化し、必要な情報を素早く取り出せるようにする

**今後のレッスンでは、プロンプトエンジニアリングでどのテクニックを選ぶべきか、より具体的で繊細なアプローチを扱います。このレッスンでは「全部盛り」のショットガンアプローチで一気に適用します。**


### Adding a system prompt

提案を1つずつ適用します。まず、アシスタントの文脈と役割を定める system prompt を作ります。経験上、system prompt に入れるべき情報は主にこの種の「役割・前提」に限るのが良いです。

このユースケースでは、Claude に「長い患者履歴を簡潔な要約に変換するのが得意な医療従事者」として振る舞わせたいです。

例として次のようにできます：


In [33]:
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

### Structuring input data

Claude を使うときの重要なコツの1つは、入力データを XML タグで明確にラベル付けすることです。現在のプロンプトでは、診療記録を次のように“そのまま”渡しています：

> I have this patient medical record. Can you summarize it for me?
> 
> {medical record goes here}
> 
> I need this for a quick review before the patient's appointment tomorrow.

ここに単純な改善として、入力データを XML タグで囲みます。`<patient_record>` タグを使ってプロンプトを更新しましょう。


In [34]:
#System Prompt
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

#Improved Prompt (still a work in progress!)
updated_prompt = """
<patient_record>
{record}
</patient_record>
"""

### Provide clear instructions

Claude から良い結果を得るには、明確で直接的な指示が不可欠です。人に初めて仕事を頼むときと同じで、何をしてほしいかを具体的に説明するほど、応答はより正確になります。

初期の「悪い」プロンプトの指示は次だけでした：

> I have this patient medical record. Can you summarize it for me?

これはあまりにも曖昧です。どんな要約が欲しいのかを、明確で具体的な指示として定義する必要があります。

例えば、次のカテゴリを要約に含めたいとします：

* 患者名
* 年齢
* 主要な診断の一覧
* 処方薬の一覧
* 薬以外の治療（理学療法、CBT など）の一覧
* 最近の懸念点の一覧
* 医師向けの重要アクション項目の一覧

また、出力形式も明確に指定することが重要です。ここでは、単なる「リスト」ではなく、箇条書き（bulleted lists）で出すよう指示します。

これを反映したプロンプトは次のようになります：


In [36]:
#System Prompt
system = """
You are a highly experienced medical professional with a specialty in translating complex patient histories into concise, actionable summaries. 
Your role is to analyze patient records, identify critical information, and present it in a clear, structured format that aids in diagnosis and treatment planning. 
Your summaries are invaluable for busy healthcare providers who need quick insights into a patient's medical history before appointments.
"""

#Improved Prompt (still a work in progress!)
updated_prompt = """
I need your help summarizing patient medical records for our team of doctors. 
We have a series of follow-up appointments tomorrow, and the doctors need quick, insightful summaries to prepare. 

Each summary should include the following elements in this order:
- The patient's name
- The patients age
- A bulleted list of key diagnoses in chronological order
- A bulleted list of medications the patient is prescribed
- A bulleted list of other treatments: non-medication treatments like CBT or physical therapy
- A short bulleted list of recent concerns
- A bulleted list of key action items to help our doctors prepare for the upcoming patient visit

<patient_record>
{record}
</patient_record>
"""

### Adding examples

例（examples）は、Claude の性能を引き上げ、望む出力へ誘導するための非常に強力な手段です。良い例をプロンプトに含めると、精度・一貫性・品質を大きく改善できます。特に、詳細度が高いタスクや、構造化出力／特定フォーマット遵守が必要なタスクに有効です。

現時点のプロンプトには例がありません。簡潔にするため例は1つだけにしますが、実務では2つ以上ある方が良いことが多いです。

例として使う診療記録は次です：

```
患者名: イーサン・ブラックウッド
年齢: 55歳
診療記録:

2010年: 年次健康診断、軽度の高血圧が認められる
  - 生活習慣の改善（食事、運動）を開始
2012年: 失業後に中等度のうつ病と診断
  - セルトラリンと認知行動療法（CBT）を開始
2014年: 新しい仕事、気分の改善を報告
  - セルトラリンを継続、CBTセッションを減らす
2015年: 軽度の腰痛、早期の変性椎間板疾患と診断
  - 理学療法と市販のNSAIDsを処方
2016年: 高血圧が悪化、リシノプリルを開始
2017年: 定期大腸内視鏡検査で良性ポリープを発見、検査中に切除
2018年: 持続的な咳を発症、胸部X線は異常なし
  - 胃食道逆流症（GERD）と診断
  - オメプラゾールを開始
2019年: 閉塞性睡眠時無呼吸症候群（OSA）と診断
  - CPAP療法を開始、エネルギーレベルの改善を報告
2020年: COVID-19パンデミック、在宅勤務
  - 不安の増加を報告、CBTセッションを再開（テレヘルス）
  - COVID-19ワクチン接種（モデルナ、2回）
2021年: 軽度の膝痛、MRIで軽度の半月板断裂を示す
  - 関節鏡手術を推奨されたが、患者は保存的治療を選択
2022年: 年次健康診断で前糖尿病（A1C: 6.1%）を示す
  - 生活習慣の改善を強化、栄養士に紹介
- GERD症状が解消されたためオメプラゾールを中止
2023年: 血液検査でPSA（前立腺特異抗原）の上昇を示す
  - 前立腺生検を実施、がんは陰性
- 膝痛が悪化、関節鏡手術に同意
2024年: 術後膝の回復: 良好、理学療法を継続
  - A1C値が改善（5.8%）、前糖尿病が解消
  - 負荷試験は正常、しかし心エコーで軽度のLVHを示す
  - 心臓保護のため低用量ACE阻害薬を開始
```

この入力に対して、ルールに沿った良い出力例は次のようになります：

```
患者名: イーサン・ブラックウッド
年齢: 55歳  

主要な診断:
- 高血圧 (2010年)
- うつ病 (2012年)
- 変性椎間板疾患 (2015年) 
- 胃食道逆流症（GERD） (2018年)
- 閉塞性睡眠時無呼吸症候群（OSA） (2019年)
- 前糖尿病 (2022年)
- 半月板断裂 (2021年)
- 左心室肥大（LVH） (2024年)

処方薬: 
- セルトラリン (うつ病)
- リシノプリル (高血圧)
- オメプラゾール (GERD) - 2022年に中止
- 低用量ACE阻害薬 (心臓保護 - 2024年)

その他の治療:
- 認知行動療法（CBT） (うつ病)
- 理学療法 (腰痛、術後膝の回復)
- CPAP療法 (OSA)
- 関節鏡膝手術 (2023年)

最近の懸念点:
- 膝痛の悪化
- PSAの上昇 (2023年) 
- 心エコーでの左心室肥大 (2024年)

アクション項目:  
- 術後膝の回復と理学療法のフォローアップ  
- PSA値と前立腺の健康をモニター
- 血圧と高血圧の管理を最適化
- LVH所見後のさらなる心臓検査の必要性を評価

```


この例の入力と出力をプロンプトに組み込みます。Claude にとってより明確になるよう、例全体を `<example>` タグで囲みます：


In [ ]:
#System Prompt
system = """
あなたは、複雑な患者履歴を簡潔で実用的な要約に変換する専門知識を持つ、経験豊富な医療専門家です。
あなたの役割は、診療記録を分析し、重要な情報を特定し、診断と治療計画に役立つ明確で構造化された形式で提示することです。
あなたの要約は、診察前に患者の医療履歴を素早く把握する必要がある忙しい医療提供者にとって非常に貴重です。
"""

#Improved Prompt (still a work in progress!)
updated_prompt = """
医師チームのために診療記録を要約する手助けをお願いします。
明日、一連のフォローアップ診察があり、医師たちは準備のために素早く洞察に富んだ要約を必要としています。

各要約には、次の順序で以下の要素を含めてください：
- 患者名
- 患者の年齢
- 時系列順の主要な診断の箇条書きリスト
- 患者に処方されている薬剤の箇条書きリスト
- その他の治療の箇条書きリスト: CBTや理学療法などの非薬物治療
- 最近の懸念点の短い箇条書きリスト
- 医師が今後の患者訪問に備えるための主要なアクション項目の箇条書きリスト

要約のフォーマット例は次のとおりです：

<example>
<patient_record>
患者名: イーサン・ブラックウッド
年齢: 55歳
診療記録:

2010年: 年次健康診断、軽度の高血圧が認められる
  - 生活習慣の改善（食事、運動）を開始
2012年: 失業後に中等度のうつ病と診断
  - セルトラリンと認知行動療法（CBT）を開始
2014年: 新しい仕事、気分の改善を報告
  - セルトラリンを継続、CBTセッションを減らす
2015年: 軽度の腰痛、早期の変性椎間板疾患と診断
  - 理学療法と市販のNSAIDsを処方
2016年: 高血圧が悪化、リシノプリルを開始
2017年: 定期大腸内視鏡検査で良性ポリープを発見、検査中に切除
2018年: 持続的な咳を発症、胸部X線は異常なし
  - 胃食道逆流症（GERD）と診断
  - オメプラゾールを開始
2019年: 閉塞性睡眠時無呼吸症候群（OSA）と診断
  - CPAP療法を開始、エネルギーレベルの改善を報告
2020年: COVID-19パンデミック、在宅勤務
  - 不安の増加を報告、CBTセッションを再開（テレヘルス）
  - COVID-19ワクチン接種（モデルナ、2回）
2021年: 軽度の膝痛、MRIで軽度の半月板断裂を示す
  - 関節鏡手術を推奨されたが、患者は保存的治療を選択
2022年: 年次健康診断で前糖尿病（A1C: 6.1%）を示す
  - 生活習慣の改善を強化、栄養士に紹介
- GERD症状が解消されたためオメプラゾールを中止
2023年: 血液検査でPSA（前立腺特異抗原）の上昇を示す
  - 前立腺生検を実施、がんは陰性
- 膝痛が悪化、関節鏡手術に同意
2024年: 術後膝の回復: 良好、理学療法を継続
  - A1C値が改善（5.8%）、前糖尿病が解消
  - 負荷試験は正常、しかし心エコーで軽度のLVHを示す
  - 心臓保護のため低用量ACE阻害薬を開始
</patient_record>


出力例: 

患者名: イーサン・ブラックウッド
年齢: 55歳  

主要な診断:
- 高血圧 (2010年)
- うつ病 (2012年)
- 変性椎間板疾患 (2015年) 
- 胃食道逆流症（GERD） (2018年)
- 閉塞性睡眠時無呼吸症候群（OSA） (2019年)
- 前糖尿病 (2022年)
- 半月板断裂 (2021年)
- 左心室肥大（LVH） (2024年)

処方薬: 
- セルトラリン (うつ病)
- リシノプリル (高血圧)
- オメプラゾール (GERD) - 2022年に中止
- 低用量ACE阻害薬 (心臓保護 - 2024年)

その他の治療:
- 認知行動療法（CBT） (うつ病)
- 理学療法 (腰痛、術後膝の回復)
- CPAP療法 (OSA)
- 関節鏡膝手術 (2023年)

最近の懸念点:
- 膝痛の悪化
- PSAの上昇 (2023年) 
- 心エコーでの左心室肥大 (2024年)

アクション項目:  
- 術後膝の回復と理学療法のフォローアップ  
- PSA値と前立腺の健康をモニター
- 血圧と高血圧の管理を最適化
- LVH所見後のさらなる心臓検査の必要性を評価
</example>

それでは、次の診療記録を同じ形式で要約してください：
<patient_record>
{record}
</patient_record>
"""

### Output XML structure

よくある戦略として、Claude に出力も XML タグで囲ませる方法があります。Claude が「Ok, here's the summary you requested!」のような前置きを付けたり、余計な内容を足したりしても、必要な部分だけを抽出しやすくなります。

ここでは、診療記録サマリーを `<summary>` タグ内に生成させるのが自然です。例の出力フォーマットを `<summary>` タグ付きにするだけで、Claude に期待形式を示せます。

現状の例は次の形です：

```
Here's an example of how we'd like the summaries formatted:

<example>
<patient_record>
Patient Name: Ethan Blackwood
Age: 55
Medical Record:

2010: Annual check-up, mild hypertension noted
  - Started on lifestyle modifications (diet, exercise)
medical record truncated to keep the text short here...
</patient_record>

Your output: 

Name: Ethan Blackwood
Age: 55  

Key Diagnoses:
- Hypertension (2010)
summary truncated to keep the text short here...
</example>
```

これを次のように変更し、出力を `<summary>` タグで囲むよう示します：

```
Here's an example of how we'd like the summaries formatted:

<example>
<patient_record>
Patient Name: Ethan Blackwood
Age: 55
Medical Record:

2010: Annual check-up, mild hypertension noted
  - Started on lifestyle modifications (diet, exercise)
medical record truncated to keep the text short here...
</patient_record>

<summary>
Name: Ethan Blackwood
Age: 55  

Key Diagnoses:
- Hypertension (2010)
summary truncated to keep the text short here...
</summary>
</example>
```

では、例セクションをこの形に更新しましょう。


In [ ]:
#System Prompt
system = """
あなたは、複雑な患者履歴を簡潔で実用的な要約に変換する専門知識を持つ、経験豊富な医療専門家です。
あなたの役割は、診療記録を分析し、重要な情報を特定し、診断と治療計画に役立つ明確で構造化された形式で提示することです。
あなたの要約は、診察前に患者の医療履歴を素早く把握する必要がある忙しい医療提供者にとって非常に貴重です。
"""

#Improved Prompt (still a work in progress!)
updated_prompt = """
医師チームのために診療記録を要約する手助けをお願いします。
明日、一連のフォローアップ診察があり、医師たちは準備のために素早く洞察に富んだ要約を必要としています。

各要約には、次の順序で以下の要素を含めてください：
- 患者名
- 患者の年齢
- 時系列順の主要な診断の箇条書きリスト
- 患者に処方されている薬剤の箇条書きリスト
- その他の治療の箇条書きリスト: CBTや理学療法などの非薬物治療
- 最近の懸念点の短い箇条書きリスト
- 医師が今後の患者訪問に備えるための主要なアクション項目の箇条書きリスト

要約のフォーマット例は次のとおりです：

<example>
<patient_record>
患者名: イーサン・ブラックウッド
年齢: 55歳
診療記録:

2010年: 年次健康診断、軽度の高血圧が認められる
  - 生活習慣の改善（食事、運動）を開始
2012年: 失業後に中等度のうつ病と診断
  - セルトラリンと認知行動療法（CBT）を開始
2014年: 新しい仕事、気分の改善を報告
  - セルトラリンを継続、CBTセッションを減らす
2015年: 軽度の腰痛、早期の変性椎間板疾患と診断
  - 理学療法と市販のNSAIDsを処方
2016年: 高血圧が悪化、リシノプリルを開始
2017年: 定期大腸内視鏡検査で良性ポリープを発見、検査中に切除
2018年: 持続的な咳を発症、胸部X線は異常なし
  - 胃食道逆流症（GERD）と診断
  - オメプラゾールを開始
2019年: 閉塞性睡眠時無呼吸症候群（OSA）と診断
  - CPAP療法を開始、エネルギーレベルの改善を報告
2020年: COVID-19パンデミック、在宅勤務
  - 不安の増加を報告、CBTセッションを再開（テレヘルス）
  - COVID-19ワクチン接種（モデルナ、2回）
2021年: 軽度の膝痛、MRIで軽度の半月板断裂を示す
  - 関節鏡手術を推奨されたが、患者は保存的治療を選択
2022年: 年次健康診断で前糖尿病（A1C: 6.1%）を示す
  - 生活習慣の改善を強化、栄養士に紹介
- GERD症状が解消されたためオメプラゾールを中止
2023年: 血液検査でPSA（前立腺特異抗原）の上昇を示す
  - 前立腺生検を実施、がんは陰性
- 膝痛が悪化、関節鏡手術に同意
2024年: 術後膝の回復: 良好、理学療法を継続
  - A1C値が改善（5.8%）、前糖尿病が解消
  - 負荷試験は正常、しかし心エコーで軽度のLVHを示す
  - 心臓保護のため低用量ACE阻害薬を開始
</patient_record>


<summary>
患者名: イーサン・ブラックウッド
年齢: 55歳  

主要な診断:
- 高血圧 (2010年)
- うつ病 (2012年)
- 変性椎間板疾患 (2015年) 
- 胃食道逆流症（GERD） (2018年)
- 閉塞性睡眠時無呼吸症候群（OSA） (2019年)
- 前糖尿病 (2022年)
- 半月板断裂 (2021年)
- 左心室肥大（LVH） (2024年)

処方薬: 
- セルトラリン (うつ病)
- リシノプリル (高血圧)
- オメプラゾール (GERD) - 2022年に中止
- 低用量ACE阻害薬 (心臓保護 - 2024年)

その他の治療:
- 認知行動療法（CBT） (うつ病)
- 理学療法 (腰痛、術後膝の回復)
- CPAP療法 (OSA)
- 関節鏡膝手術 (2023年)

最近の懸念点:
- 膝痛の悪化
- PSAの上昇 (2023年) 
- 心エコーでの左心室肥大 (2024年)

アクション項目:  
- 術後膝の回復と理学療法のフォローアップ  
- PSA値と前立腺の健康をモニター
- 血圧と高血圧の管理を最適化
- LVH所見後のさらなる心臓検査の必要性を評価
</summary>
</example>

それでは、次の診療記録を同じ形式で要約してください：

<patient_record>
{record}
</patient_record>
"""

### Recap of the prompt changes

新しいプロンプトを試す前に、今回の変更点を整理します：

* system prompt を追加し、文脈と役割を明確化した
* 入力データを XML タグ（ここでは `<patient_record>`）で構造化し、入力範囲を明確にした
* 要約が注目すべき項目（診断、薬、最近の懸念など）を具体的に指示した
* 望ましい出力フォーマットを指定した
* 例（入力＋理想出力）を追加して、返答をガイドした
* 出力を `<summary>` タグ内に生成するよう依頼した

---


## Testing out the new prompt

新しい `system` と `updated_prompt` を使って、同じ5件の診療記録（`patient_records`）で試します：


In [46]:
def generate_summary_with_improved_prompt(patient_record):
    prompt_with_record = updated_prompt.format(record=patient_record) #use our rewritten prompt!
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": prompt_with_record}]
    )
    print("===============================")
    print(response.content[0].text)

In [47]:
generate_summary_with_improved_prompt(patient_records[0])

<summary>
Name: Evelyn Thompson
Age: 78

Key Diagnoses:
- Type 2 Diabetes (1985)
- Hypertension (1992)
- Osteoarthritis (Hip and Knee Replacements in 1998 and 2017)
- Hypothyroidism (2000)
- Atrial Fibrillation (2005)
- Vitamin B12 Deficiency (2008)
- Transient Ischemic Attack (TIA) (2011)
- Breast Cancer (2013)
- Chronic Kidney Disease (CKD) Stage 3 (2015)
- Pneumonia (2018)
- Mild Cognitive Impairment (2019)
- Recurrent Urinary Tract Infections (UTIs) (2021)

Medications:
- Metformin (Diabetes)
- Lisinopril (Hypertension)
- Levothyroxine (Hypothyroidism)
- Warfarin (Atrial Fibrillation)
- Aspirin (Antiplatelet)
- Anastrozole (Breast Cancer Recurrence Prevention)
- Donepezil (Cognitive Impairment)
- Low-dose Prophylactic Antibiotics (Recurrent UTIs)

Other Treatments:
- Total Hip Replacement (1998)
- Cataract Surgery (2003)
- Vitamin B12 Injections (2008)
- Lumpectomy and Radiation (Breast Cancer, 2013)
- Total Knee Replacement (2017)
- Physical Therapy and Home Health Aide (2023)

Re

In [48]:
generate_summary_with_improved_prompt(patient_records[1])

<summary>
Name: Marcus Reyes
Age: 42

Key Diagnoses:
- Generalized Anxiety Disorder (GAD) (2001)  
- Major Depressive Disorder (MDD) (2003)
- Attention-Deficit/Hyperactivity Disorder (ADHD) (2007)
- Obstructive Sleep Apnea (OSA) (2012)
- Hypertension (2014)
- Type 2 Diabetes (2016)
- Gastroesophageal Reflux Disease (GERD) (2019)
- Plantar Fasciitis (2021)

Medications:
- Paroxetine (GAD) - switched to escitalopram in 2009
- Bupropion (MDD) - switched to venlafaxine in 2018  
- Methylphenidate (ADHD)
- Losartan (hypertension)
- Metformin (diabetes)
- Insulin therapy (diabetes) - initiated in 2017
- Omeprazole (GERD)  
- Quetiapine (antipsychotic) (2023)
- Lamotrigine (mood stabilizer) (2023)

Other Treatments: 
- Cognitive Behavioral Therapy (CBT)
- Dialectical Behavior Therapy (DBT) (2020)
- CPAP therapy (OSA)
- Orthotics and physical therapy (plantar fasciitis)
- Inpatient psychiatric treatment (2023)

Recent Concerns:
- Suicide attempt (2023)  
- Panic attack (2022)
- Weight gain and

In [49]:
generate_summary_with_improved_prompt(patient_records[2])

<summary>
Name: Lily Chen
Age: 8

Key Diagnoses:
- Tetralogy of Fallot (TOF) (2016 - Birth)
- Developmental Delay (2018)
- Failure to Thrive (2019)  
- Sensory Processing Disorder (SPD) (2020)
- Attention-Deficit/Hyperactivity Disorder (ADHD) (2022)
- Sensorineural Hearing Loss (2023)

Medications:
- Amiodarone (post-operative arrhythmias - 2017)

Other Treatments:
- Surgical Repair of TOF (2017)
- Early Intervention Services (2018) 
- Speech Therapy (2018 - ongoing)
- Occupational Therapy (2019 - 2024)  
- Integrated Preschool Program with IEP (2021)
- Behavioral Therapy for ADHD (2022)
- Cochlear Implant Surgery (2023)
- Mainstream School with IEP Support (2023 - ongoing)
- Balloon Valvuloplasty for Pulmonary Stenosis (2022)

Recent Concerns: 
- Progression of Pulmonary Stenosis requiring intervention (2022)
- Ongoing speech development delays

Action Items:
- Monitor cardiac status, pulmonary valve function
- Continue speech and language therapy  
- Assess need for ongoing ADHD trea

In [50]:
generate_summary_with_improved_prompt(patient_records[3])

<summary>
Name: Jason Tran
Age: 25

Key Diagnoses:
- Asthma (2010)
- Concussions (2012, 2014)
- Post-Concussion Syndrome (2014)
- Depression and Anxiety (2015)
- ACL Tear (2016)
- PTSD (2018)
- Sleep Apnea (2021)

Medications:
- Albuterol inhaler (asthma)
- Fluoxetine (depression, anxiety)

Other Treatments:
- Cognitive Rehabilitation Therapy (post-concussion syndrome)
- Cognitive-Behavioral Therapy (CBT) (PTSD, anxiety)
- Exposure Therapy (PTSD)
- Mindfulness Meditation and Yoga (anxiety)
- CPAP Therapy (sleep apnea)
- Surgical Fixation (fractured tibia, 2013)
- ACL Reconstruction Surgery (2016)
- Physical Rehabilitation (post-ACL surgery)

Recent Concerns:
- Returning to sports and managing anxiety/PTSD
- Sleep quality and sleep apnea

Action Items:
- Monitor asthma control and medication adherence
- Continue CBT and exposure therapy for PTSD/anxiety
- Assess need for ongoing CPAP therapy for sleep apnea
- Encourage gradual return to low-impact sports and training
- Support youth con

In [51]:
generate_summary_with_improved_prompt(patient_records[4])

<summary>
Name: Amira Khan
Age: 36

Key Diagnoses:
- Fibromyalgia (2011)
- Cutaneous Lupus Erythematosus (CLE) (2012)
- Systemic Lupus Erythematosus (SLE) (2013)
- Lupus Nephritis (2014)
- Interstitial Lung Disease (ILD) (2015)
- Sjögren's Syndrome (2016)
- Lupus Cerebritis (2017)
- Avascular Necrosis (AVN) of Hip (2018)
- Steroid-induced Diabetes (2021)
- Diastolic Dysfunction (2023)

Medications:
- Pregabalin (fibromyalgia)
- Hydroxychloroquine (SLE)
- Prednisone (SLE)
- Mycophenolate Mofetil (lupus nephritis)
- Cyclophosphamide (ILD)
- Pilocarpine (Sjögren's syndrome)
- Rituximab (lupus cerebritis)
- Belimumab (reduce flare frequency)
- Metformin and Insulin (steroid-induced diabetes)
- Abatacept (SLE flare)
- ACE Inhibitors (diastolic dysfunction)

Other Treatments:
- Physical Therapy (fibromyalgia, joint health)
- Topical Corticosteroids and Sunscreen (CLE)
- Home Oxygen Therapy (ILD)
- Total Hip Replacement Surgery (AVN)
- Yoga (joint health)
- Cardio-rehab (diastolic dysfunction

うまく動いています！ 出力は、指定したルールとフォーマットに沿って一貫しています。

上にスクロールして、初期プロンプトでの出力と、改善後の一貫した出力を比較してみてください。

---


## Switching things up: JSON!

現在のプロンプトで生成されるテキスト要約でも十分な場合はありますが、実務ではプログラムで扱いやすい **構造化出力** が欲しいことが多いです。最も一般的なのは JSON です。

**Note: JSON を「強制」する一番簡単な方法は、Claude の tool use（ツール利用）機能を使うことです（別コースの tool-use レッスンで扱います： https://github.com/anthropics/courses/blob/master/tool_use/03_structured_outputs.ipynb ）。ここでは、プロンプト変更が出力に与える影響を示すために、あえてプロンプトだけで調整します。**

JSON 出力に合わせるために必要な変更：
* Claude に JSON 形式で返すよう明示し、JSON に含める項目を指定する
* 例のサマリー出力を JSON 形式に更新する


In [ ]:
# Unchanged System Prompt
system = """
あなたは、複雑な患者履歴を簡潔で実用的な要約に変換する専門知識を持つ、経験豊富な医療専門家です。
あなたの役割は、診療記録を分析し、重要な情報を特定し、診断と治療計画に役立つ明確で構造化された形式で提示することです。
あなたの要約は、診察前に患者の医療履歴を素早く把握する必要がある忙しい医療提供者にとって非常に貴重です。
"""

#Prompt updated to generate JSON
updated_json_prompt = """
医師チームのために診療記録を要約する手助けをお願いします。
明日、一連のフォローアップ診察があり、医師たちは準備のために素早く洞察に富んだ要約を必要としています。

以下の構造でJSON形式で要約を提供してください：
{{
  "name": "患者のフルネーム",
  "age": 患者の年齢（整数）,
  "key_diagnoses": [
    {{
      "diagnosis": "主要な診断",
      "year": 診断年（整数）
    }},
    ...
  ],
  "medications": [
    {{
      "name": "薬剤名",
      "purpose": "用途の簡潔な説明"
    }},
    ...
  ],
  "other_treatments": [
    {{
      "treatment": "治療名",
      "purpose": "用途の簡潔な説明"
    }},
    ...
  ],
  "recent_concerns": [
    "最近の健康問題や懸念点の簡潔な記述"
  ],
  "action_items": [
    "アクション項目1",
    "アクション項目2",
    ...
  ]
}}

要約のフォーマット例は次のとおりです：

<example>
<patient_record>
患者名: イーサン・ブラックウッド
年齢: 55歳
診療記録:

2010年: 年次健康診断、軽度の高血圧が認められる
  - 生活習慣の改善（食事、運動）を開始
2012年: 失業後に中等度のうつ病と診断
  - セルトラリンと認知行動療法（CBT）を開始
2014年: 新しい仕事、気分の改善を報告
  - セルトラリンを継続、CBTセッションを減らす
2015年: 軽度の腰痛、早期の変性椎間板疾患と診断
  - 理学療法と市販のNSAIDsを処方
2016年: 高血圧が悪化、リシノプリルを開始
2017年: 定期大腸内視鏡検査で良性ポリープを発見、検査中に切除
2018年: 持続的な咳を発症、胸部X線は異常なし
  - 胃食道逆流症（GERD）と診断
  - オメプラゾールを開始
2019年: 閉塞性睡眠時無呼吸症候群（OSA）と診断
  - CPAP療法を開始、エネルギーレベルの改善を報告
2020年: COVID-19パンデミック、在宅勤務
  - 不安の増加を報告、CBTセッションを再開（テレヘルス）
  - COVID-19ワクチン接種（モデルナ、2回）
2021年: 軽度の膝痛、MRIで軽度の半月板断裂を示す
  - 関節鏡手術を推奨されたが、患者は保存的治療を選択
2022年: 年次健康診断で前糖尿病（A1C: 6.1%）を示す
  - 生活習慣の改善を強化、栄養士に紹介
- GERD症状が解消されたためオメプラゾールを中止
2023年: 血液検査でPSA（前立腺特異抗原）の上昇を示す
  - 前立腺生検を実施、がんは陰性
- 膝痛が悪化、関節鏡手術に同意
2024年: 術後膝の回復: 良好、理学療法を継続
  - A1C値が改善（5.8%）、前糖尿病が解消
  - 負荷試験は正常、しかし心エコーで軽度のLVHを示す
  - 心臓保護のため低用量ACE阻害薬を開始
</patient_record>


<summary>
{{
  "name": "イーサン・ブラックウッド",
  "age": 55,
  "key_diagnoses": [
    {{
      "diagnosis": "高血圧",
      "year": 2010
    }},
    {{
      "diagnosis": "うつ病",
      "year": 2012
    }},
    {{
      "diagnosis": "変性椎間板疾患",
      "year": 2015
    }},
    {{
      "diagnosis": "胃食道逆流症（GERD）",
      "year": 2018
    }},
    {{
      "diagnosis": "閉塞性睡眠時無呼吸症候群（OSA）",
      "year": 2019
    }},
    {{
      "diagnosis": "前糖尿病",
      "year": 2022
    }},
    {{
      "diagnosis": "左心室肥大（LVH）",
      "year": 2024
    }}
  ],
  "medications": [
    {{
      "name": "リシノプリル",
      "purpose": "高血圧管理"
    }},
    {{
      "name": "セルトラリン",
      "purpose": "うつ病"
    }},
    {{
      "name": "ACE阻害薬（低用量）",
      "purpose": "心臓保護"
    }}
  ],
  "other_treatments": [
    {{
      "treatment": "認知行動療法（CBT）",
      "purpose": "うつ病管理"
    }},
    {{
      "treatment": "理学療法",
      "purpose": "腰痛と膝の回復"
    }},
    {{
      "treatment": "CPAP療法",
      "purpose": "閉塞性睡眠時無呼吸症候群"
    }},
    {{
      "treatment": "関節鏡膝手術",
      "purpose": "半月板断裂の修復"
    }}
  ],
  "recent_concerns": [
    "膝痛の悪化、手術で対処",
    "PSAの上昇（2023年）、生検は陰性",
    "軽度の左心室肥大（2024年）"
  ],
  "action_items": [
    "術後膝の回復と理学療法の進捗をフォローアップ",
    "PSA値と前立腺の健康をモニター",
    "LVH所見後の心臓の健康状態を評価",
    "血圧管理を確認"
  ]
}}
</summary>
</example>

それでは、次の診療記録を同じ形式で要約してください。JSON要約は<summary>タグ内に出力してください。
"""

#Broke this part into its own variable to make it easier to use the `format` method later
# (the JSON curly braces in the above prompt variable cause problems when using `format` )
medical_record_input_prompt = """
<patient_record>
{record}
</patient_record>
"""

次の関数で試してみましょう：


In [72]:
def generate_summary_with_json_prompt(patient_record):
    final_prompt_part = medical_record_input_prompt.format(record=patient_record) #add the medical record to the final prompt piece
    complete_prompt = updated_json_prompt + final_prompt_part
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": complete_prompt}]
    )
    print(response.content[0].text)

In [73]:
generate_summary_with_json_prompt(patient_records[0])

<summary>
{
  "name": "Evelyn Thompson",
  "age": 78,
  "key_diagnoses": [
    {
      "diagnosis": "Type 2 Diabetes",
      "year": 1985
    },
    {
      "diagnosis": "Hypertension",
      "year": 1992
    },
    {
      "diagnosis": "Osteoarthritis",
      "year": 1998
    },
    {
      "diagnosis": "Hypothyroidism",
      "year": 2000
    },
    {
      "diagnosis": "Atrial Fibrillation",
      "year": 2005
    },
    {
      "diagnosis": "Vitamin B12 Deficiency",
      "year": 2008
    },
    {
      "diagnosis": "Transient Ischemic Attack (TIA)",
      "year": 2011
    },
    {
      "diagnosis": "Breast Cancer (Stage 2)",
      "year": 2013
    },
    {
      "diagnosis": "Chronic Kidney Disease (CKD Stage 3)",
      "year": 2015
    },
    {
      "diagnosis": "Mild Cognitive Impairment",
      "year": 2019
    }
  ],
  "medications": [
    {
      "name": "Metformin",
      "purpose": "Diabetes management"
    },
    {
      "name": "Lisinopril",
      "purpose": "Hypertensi

OK、望んだ JSON が得られています！

この JSON 出力がどれだけ扱いやすいかを示すために、複数患者のサマリーをまとめて生成し、医師が当日の診察で必要になりそうな「action items」だけを抽出して一覧化する関数を書いてみましょう。


In [97]:
import re
import json

def generate_summary_with_json_prompt(patient_record):
    final_prompt_part = medical_record_input_prompt.format(record=patient_record) #add the medical record to the final prompt piece
    complete_prompt = updated_json_prompt + final_prompt_part
    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        system=system, #add in our system prompt!
        messages=[{"role": "user", "content": complete_prompt}]
    )
    return response.content[0].text

def extract_json_from_summary_tags(summary_string):
     # Use regular expressions to find content between <summary> tags
    match = re.search(r'<summary>\s*(.*?)\s*</summary>', summary_string, re.DOTALL)
    if not match:
        print("No <summary> tags found in the input string.")
        return

    # Extract the JSON string
    return  match.group(1)

def extract_action_items(model_response):
    summary = extract_json_from_summary_tags(model_response)
    try:
        # Parse the JSON string
        summary_data = json.loads(summary)

        # Extract and print action items
        action_items = summary_data.get('action_items', [])
        patient_name = summary_data.get('name', "")
        print(f"Action Items for {patient_name}:")
        for idx, item in enumerate(action_items, 1):
            print(f"{idx}. {item}")
    except json.JSONDecodeError:
        print("Failed to parse JSON. Make sure the content between <summary> tags is valid JSON.")

def generate_daily_action_items(patient_records):
    # Takes a list of patient records, generates a summary for each, and prints the action items:
    for record in patient_records:
        summary = generate_summary_with_json_prompt(record)
        extract_action_items(summary)

In [96]:
generate_daily_action_items(patient_records)

Action Items for Evelyn Thompson:
1. Monitor kidney function and adjust medications as needed
2. Evaluate cognitive status and optimize treatment
3. Assess mobility and home safety, consider assistive devices
4. Follow up on breast cancer surveillance and prevention
Action Items for Marcus Reyes:
1. Monitor mental health closely, ensure adequate support system
2. Evaluate effectiveness of updated medication regimen
3. Continue lifestyle and diabetes management (diet, exercise)
4. Follow-up physical therapy for plantar fasciitis
5. Provide resources for patient and family related to mental health
Action Items for Lily Chen:
1. Evaluate current educational and therapy needs
2. Monitor hearing and speech progress post-cochlear implant
3. Assess cardiac status and need for future interventions
4. Continue close developmental monitoring and support
Action Items for Jason Tran:
1. Monitor asthma control and medication adherence
2. Assess mental health status and therapy progress
3. Evaluate 